Problem<br><br>
There is a T-shirt manufacturer. They produce two kinds of T-shirts. One is a T-shirt that is made of silk. The other is one that is made of cotton. Silk T-shirts per item sells for $45 and costs $30 to make. The cotton T-shirt per item sells for $20 and costs $10. To meet customer demand, at least 2000 units of silk T-shirts and at least 8000 units of cotton T-shirts need to be produced and sold. The manufacturer can produce 8000 units of two kinds of T-shirts at most. What is the maximum revenue that the manufacture can make?

In [3]:
from pulp import *
import pandas as pd

In [4]:
df = pd.DataFrame({'Variable': ['silk', 'cotton'],
                   'Price': [45, 20],
                   'Cost': [30, 10],
                   'Demand': [2000, 8000]
                    })


df['Margin'] = df['Price'] - df['Cost']

print(df)

  Variable  Price  Cost  Demand  Margin
0     silk     45    30    2000      15
1   cotton     20    10    8000      10


Easy Setup

In [5]:
# Initialize the LP
prob = LpProblem('T-shirt Manufacturer', LpMaximize)

# Decision Variables
x = LpVariable.dicts('x', [1,2], lowBound=0, cat='Integer')  # silk, cotton

# Objective Function
prob += 15 * x[1] + 10 * x[2] # Max Revenue

# Add Constraints
prob += x[1] + x[2] <= 8000, 'Total Demand'
prob += x[1] <= 2000, 'Silk Demand'
prob += x[2] <= 8000, 'Cotton Demand'
print(prob)

# Solve the problem
status = prob.solve()
print(LpStatus[status])

#Display variables
for var in prob.variables():
    print(f"{var.name} = {var.varValue}")

#Display objective
max_revenue = value(prob.objective) # 90000
print(f"max revenue = {max_revenue}")

T-shirt_Manufacturer:
MAXIMIZE
15*x_1 + 10*x_2 + 0
SUBJECT TO
Total_Demand: x_1 + x_2 <= 8000

Silk_Demand: x_1 <= 2000

Cotton_Demand: x_2 <= 8000

VARIABLES
0 <= x_1 Integer
0 <= x_2 Integer

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jason/Documents/GitHub/naut/quantitative_analysis/env/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fb/p1td_8397cg27vvh9njm_gtm0000gn/T/724b8fde8b494c69995cde4fe15b3871-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/fb/p1td_8397cg27vvh9njm_gtm0000gn/T/724b8fde8b494c69995cde4fe15b3871-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 19 RHS
At line 23 BOUNDS
At line 26 ENDATA
Problem MODEL has 3 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 90000 - 0.00 seconds
Cgl0004I processed model has 1 r

/Users/jason/Documents/GitHub/naut/quantitative_analysis/env/lib/python3.10/site-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Hard Setup

In [7]:
# Initialize the LP
prob = LpProblem('T-shirt Manufacturer', LpMaximize)

inv_items = list(df['Variable']) # List of variable names

margin = dict(zip(inv_items, df['Margin'])) # Dictionary of Margin

demand = dict(zip(inv_items, df['Demand'])) # Dictionary of Demand

# Decision Variables
inv_vars = LpVariable.dicts('Variable', inv_items, lowBound=0, cat='Integer')

# Objective Function
prob += lpSum([inv_vars[i] * margin[i] for i in inv_items])

# Constraints
prob += lpSum([inv_vars[i] for i in inv_items]) <= 8000, 'Total Demand'
prob += inv_vars['silk'] <= 2000, 'Silk Demand'
prob += inv_vars['cotton'] <= 8000, 'Cotton Demand'

# Solve the problem
status = prob.solve()
print(LpStatus[status])

# Objective
max_revenue = value(prob.objective) # 90000
print(max_revenue)

# Variables' values
print('Materials needed\n' + '-' * 69)

for var in prob.variables():
    if var.varValue >= 0:
       print(var.name, '=', var.varValue)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jason/Documents/GitHub/naut/quantitative_analysis/env/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fb/p1td_8397cg27vvh9njm_gtm0000gn/T/f93aa2bb44264c97b9759731db78fdfa-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/fb/p1td_8397cg27vvh9njm_gtm0000gn/T/f93aa2bb44264c97b9759731db78fdfa-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 19 RHS
At line 23 BOUNDS
At line 26 ENDATA
Problem MODEL has 3 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 90000 - 0.00 seconds
Cgl0004I processed model has 1 rows, 2 columns (2 integer (0 of which binary)) and 2 elements
Cutoff increment increased from 1e-05 to 4.9999
Cbc0012I Integer solution of -90000 found by DiveCoefficient after 0 iterations and 